In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten, Dense, Dropout

In [ ]:
Img_size = [224, 224]

In [ ]:
import matplotlib.pyplot as plt
import os
import random

path_1 = ["fractured", "not_fractured"]
random_path=random.choice(path_1)

# Klasör yolunu belirtin
klasor_yolu = f'C:\\Users\\batuy\Documents\\VS Code\\Bone_Fracture_Binary_Classification\\train\\{random_path}'

# Klasördeki tüm dosyaları alın
dosya_listesi = os.listdir(klasor_yolu)

resimler = []
for dosya in dosya_listesi:
    if dosya.endswith('.png') or dosya.endswith('.jpg'):
        resimler.append(dosya)

random_img = random.sample(resimler, 5)

# Resimleri görselleştirin
for resim in random_img:
    print("Path: ", random_path)
    resim_yolu = os.path.join(klasor_yolu, resim)
    img = plt.imread(resim_yolu)
    plt.imshow(img)
    plt.title(resim)  
    plt.axis('off')  
    plt.show()


In [ ]:
train_data = ImageDataGenerator(shear_range=0.1,
                                zoom_range=0.3,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1./255,)

test_data = ImageDataGenerator(rescale=1./255)
val_data = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_data.flow_from_directory(
    directory="C:\\Users\\batuy\Documents\\VS Code\\Bone_Fracture_Binary_Classification\\train",
    target_size=Img_size,
    class_mode="binary",
    batch_size=64
)

In [ ]:
test_data_gen = train_data.flow_from_directory(
    directory="C:\\Users\\batuy\Documents\\VS Code\\Bone_Fracture_Binary_Classification\\test",
    target_size=Img_size,
    class_mode="binary",
    batch_size=64
)

In [ ]:
val_data_gen = train_data.flow_from_directory(
    directory="C:\\Users\\batuy\Documents\\VS Code\\Bone_Fracture_Binary_Classification\\val",
    target_size=Img_size,
    class_mode="binary",
    batch_size=64
)

In [ ]:
model_vgg = VGG16(input_shape=Img_size + [3], weights="imagenet", include_top=False)

In [ ]:
for layer in model_vgg.layers:
    layer.trainble = False

In [ ]:
x = model_vgg.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(520, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(124, activation="relu")(x)
prediction = Dense(1, activation="softmax")(x)

In [ ]:
model = Model(inputs = model_vgg.input, outputs=prediction)

In [ ]:
callbacks_1 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, min_lr=0.000001, patience=2)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
fit_ = model.fit(train_data_gen, validation_data=val_data_gen, batch_size=928, epochs=50, callbacks=callbacks_1)